# Decoding

If a notebook is imported, it should provide a natural __traceback__ experience similar to python imports.  The `importnb` importer creates a just decoder object that retains line numbers to the raw json when the notebook is imported.

In [1]:
    from json.decoder import JSONObject, JSONDecoder, WHITESPACE, WHITESPACE_STR    
    class LineNoDecoder(JSONDecoder):
        """A JSON Decoder to return a dict with lines numbers in the metadata."""
        def __init__(self, *, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, strict=True, object_pairs_hook=None):
            from json.scanner import py_make_scanner    
            super().__init__(object_hook=object_hook, parse_float=parse_float, parse_int=parse_int, parse_constant=parse_constant, strict=strict, 
                             object_pairs_hook=object_pairs_hook)
            self.parse_object = self.object
            self.scan_once = py_make_scanner(self)
            
        def object(
            self, 
            s_and_end, 
            strict, scan_once, object_hook, object_pairs_hook, memo=None, _w=WHITESPACE.match, _ws=WHITESPACE_STR
        ) -> (dict, int):
            object, next = JSONObject(s_and_end, strict, scan_once, object_hook, object_pairs_hook, memo=memo, _w=_w, _ws=_ws)

            if 'cell_type' in object: 
                object['metadata'].update({
                    'lineno':  len(s_and_end[0][:next].rsplit('"source":', 1)[0].splitlines())
                })
                
            for key in ('source', 'text'): 
                if key in object: object[key] = ''.join(object[key])
            return dict(object), next

# Compilation

Compilation occurs in the __3__ steps:

1. Text is transformed into a valid source string.
2. The source string is parsed into an abstract syntax tree
3. The abstract syntax compiles to valid bytecode 

In [12]:
    import ast, sys
    from json import load, loads
    from pathlib import Path
    from textwrap import dedent

In [11]:
    from codeop import Compile
    from dataclasses import dataclass, field
    class Compiler(Compile):
        """{Shell} provides the IPython machinery to objects."""
        filename: str = '<Shell>'            
            
        def ast_transform(Compiler, node): return node
        
        @property
        def transform(Compiler): return dedent

        def compile(Compiler, ast): return compile(ast, Compiler.filename, 'exec')
                
        def ast_parse(Compiler, source, filename='<unknown>', symbol='exec', lineno=0): 
            return ast.increment_lineno(ast.parse(source, Compiler.filename, 'exec'), lineno)

In [5]:
    class NotebookExporter:
        def from_filename(self, filename, *args,  **dict):
            with open(filename, 'r') as file_stream:
                return self.from_file(file_stream)

In [6]:
    NotebookNode = dict

In [10]:
    if __name__ ==  '__main__':
        from pathlib import Path
        from nbconvert.exporters.script import ScriptExporter
        Path('compiler_python.py').write_text(ScriptExporter().from_filename('compiler_python.ipynb')[0])
        __import__('doctest').testmod()